# Take Home Project: Wrangling FERC Form 1

* This task is an example of the kind of work we do to make public energy data usable for analysis.
* We want to be able to explore your general approach together and see how you think about these kinds of problems.
* **Spend 2-4 hours working on it.** This doesn't have to happen all at once. We want you to have time to play with the data, step away from it to think, and then come back to it again.
* Feel free to use whatever documentation or online resources you would normally consult while working on a data wrangling problem.
* Feel free to use additional 3rd party libraries if you want to.  You should be able to install them from within the notebook using `!pip install packagename` or `!conda install packagename`

## Email us your notebook within a week.
* Send it to [hello@catalyst.coop](mailto:hello@catalyst.coop) (normally we'd have you make a PR but... we don't want everyone looking at each others solutions)
* We'll review your notebook and if it looks good, we'll reach out to schedule a longer conversation about it, and another technical interview.

## Some questions to keep in mind:
* What assumptions are you making about the data?
* Is the raw data well structured?
* How will you test whether / when those assumptions are valid?
* How would you / did you deal with the data that don’t conform to those assumptions?
* If there are records which can’t be reasonably cleaned automatically, but were high value in an advocacy context, how would you integrate manual cleaning into the automated process so that the manual effort is captured, and can be incrementally improved over time?
* What expectations do you have about the output data?
* What kind of data validation checks would you design to make sure that the output meets your expectations? These could be either integrated into the table transformation process, or run on the final output.
* How do you decide when data isn’t recoverable?
* How will you evaluate the completeness of the data that you’ve been able to extract?
* What kind of queries are you trying to make easy with the structure of the output data?
* What parts of this process might make sense to generalize / abstract for re-use in extracting, cleaning, and reorganizing data from other tables?

# Background on the FERC Form 1 Database
* The FERC Form 1 collects financial data about electric utilities in the US. It’s a treasure-trove of information if you want to understand how these utilities make and spend money. The capital they have locked up in existing fossil fuel infrastructure is one of the big reasons they fight against the transition to clean energy. Data from the FERC Form 1 can help advocates understand which utilities will be easiest to engage in the transition, and which ones may be hopeless pyromaniacs.
* Unfortunately, FERC does not organize its data very well, or do much quality control, so this data is difficult to extract and use. We’ve built a script that pulls together all of FERC’s annual Visual FoxPro databases into a single SQLite database covering all the years of data. Then we write extract and transform functions to pull tables from this multi-year DB and clean them up for easier analysis.
* To help us understand how you approach working with messy data and turning it into something usable, we’d like you to develop a strategy for reshaping and cleaning the data in one of these tables.
* [Here is some documentation about the FERC Form 1 Database](https://catalystcoop-pudl.readthedocs.io/en/dev/data_sources/ferc1_db_notes.html), including a mapping between database tables and the pages of the PDF that their data is collected from.

# Set up access to the FERC Form 1 DB
* You can download a copy of our FERC Form 1 SQLite DB from: https://data.catalyst.coop/ferc1.db
* Substitute the path to that file on your system below:

In [ ]:
import sqlalchemy as sa
import pandas as pd

FERC1_DB_PATH = "../data/ferc1.sqlite"

ferc1_engine = sa.create_engine(f"sqlite:///{FERC1_DB_PATH}")

# Prepare the FERC Form 1 Small Plants table for quantitative analysis.
* Explore the Small Plants table (named `f1_gnrt_plant` in the FERC 1 DB).
* Refer to the [blank Form 1 (PDF)](https://catalystcoop-pudl.readthedocs.io/en/dev/_downloads/6a316a949a522f595e7575b6fd7034b8/ferc1_blank_2022-11-30.pdf) (pages 410-411) for more context about the table.
* Our goal is to make as much of the information as possible available for easy programmatic analysis.
* Unfortunately, in its raw form this data is only semi-structured.

## Identify issues in the data
* Make a list of issues that would need to be addressed before this table would be ready for analytical use.
* Show us how you identified the issues you highlight, and briefly talk through why they're problematic, and how you might approach fixing them.
* Don't worry about cataloging every possible issue, but do try to identify several of the biggest problems.

## Tackle one or more of the issues you identified
* Choose one or more of the major issues you identified above, and address it in Python, using pandas and whatever other packages you find useful.
* Imagine these being the first few steps of an ETL pipeline that would ultimately output a tidy, well-structured, analysis-ready database table.

In [ ]:
# Read the small plants table, ignoring footnote reference columns:
small_plants = pd.read_sql("f1_gnrt_plant", ferc1_engine)
small_plants = small_plants.loc[:, ~small_plants.columns.str.endswith("_f")]

In [ ]:
small_plants.info(verbose=True)

In [ ]:
# Assumption that early data is representative
# Assumption that data is well/evenly distributed
small_plants.head(50)

## Issues Identified

### Extraneous Data
Many of the rows in the table, from the samples I've looked at, appear to not actually correspond to a single small plant. Some appear to be headings, which introduce a section of small plant rows that follow. While others appear to be an aggregation row, referring to some cumulative information about the rows preceding it. The _yr_constructed_ column appears to be the most reliable indicator of whether a row actually represents a small plant or some extraneous information. However, the rows that do not represent a single row still have useful information that can be used to supplement the rows in the same section that might have missing information. For example for the rows with _respondent_id_ 115, the first row has the value "Hydro" in the _plant_name_ column, followed by five rows (2-6), that actually represent small plants, evidenced by the values we can see in those rows, including a valid _yr_constructed_ value. However each plant referenced in those valid rows are missing a value in their _kind_of_fuel_ column. It might be appropriate in this scenario to treat row 1 with the value "Hydro" as an introduction row for the small hydro plants that follow. So in cleaning up and transforming this table into a more usable format, I would look for other patterns like this in the data, so that we could use the "heading" rows to fill in the gaps of information missing in the valid rows. In this case, we could put "hydro" in the _kind_of_fuel_ column for rows 2-6. We see a similar pattern from the same _respondent_id_ 6 section in rows 25-27, where these are likely hydro electric plants.

### Missing Data and Invalid Data
Many of the rows in the table are simply missing values for many of the columns. The short script below looks for columns with missing or invalid data. The output has been copied to the table below for convenience and readability.


| Column Name      | Percent of Rows Missing Value |
| ----------- | ----------- |
| fuel_cost | 89.59% |
| expns_fuel   | 83.34% |
| kind_of_fuel | 64.84% |
| row_prvlg | 58.68% |
| operation | 45.11% |
| expns_maint | 40.93% |
| net_generation | 38.97% |
| plant_cost_mw | 37.23% |
| plant_cost | 31.09% |
| yr_constructed | 29.40% |
| net_demand | 4.58% |
| capacity_rating | 3.32% |
| plant_name | 1.76% |
| respondent_id | 0.00% |
| report_year | 0.00% |
| spplmnt_num | 0.00% |
| row_number | 0.00% |
| row_seq | 0.00% |
| report_prd | 0.00% |

In [ ]:
# These columns should have non-NaN values based on their Python types.
NUMERICAL_COLUMNS = [
    'respondent_id',
    'report_year',
    'spplmnt_num',
    'row_number',
    'row_seq',
    'capacity_rating',
    'net_demand',
    'net_generation',
    'plant_cost',
    'plant_cost_mw',
    'operation',
    'expns_fuel',
    'expns_maint',
    'fuel_cost',
    'report_prd'
]

# Good indicator of whether a row actually corresponds to a plant
# rather than being extraneous information.
YEAR_CONSTRUCTED_COL = 'yr_constructed'

# Assumption that the value "None" is not valid for this column.
PLANT_NAME_COL = 'plant_name'
PLANT_NAME_NULL = 'none'

import re
# Simple pattern for identifying valid year entries in "yr_constructed" column.
# 29.19% of rows report no value in this column.
# 0.21% report invalid (via pattern below) values.
PATTERN = re.compile("^[0-9]{4}$")

def cell_is_invalid(row, column):
    # Plant names should not be empty or "None"
    if column == PLANT_NAME_COL:
        plant_name = row[column].strip().lower()
        return plant_name == PLANT_NAME_NULL or \
                not plant_name
    elif column == YEAR_CONSTRUCTED_COL:
        # 29.19% empty values
        # 0.21% invalid values
        # Year constructed should be a valid
        # 4 digit number only
        value = row[column].strip() 
        return not value or not PATTERN.match(value)
    elif column in NUMERICAL_COLUMNS:
        # For numerical columns we can check if
        # the value is NaN 
        return pd.isna(row[column])
    else:
        # Catch all, to make sure the value is non-empty
        return not row[column].strip()

In [ ]:
def get_missing_columns_report(df):
    # Get the number of rows in table so we have
    # the denominator when we calculate percentage below.
    total = df.shape[0]
    
    # Dictionary that maps a column name to the computed
    # percent of rows missing a valid value for that column. 
    percent_missing = {}

    # For every column, iterate through each row and determine
    # if the row has a valid value for the column. Aggregate the
    # count of invalid rows and calculate which percent of rows
    # are missing a valid value, using the validation function above.
    for col in df.columns:
        nulls = 0
        percent_missing[col] = 0
        for index, row in df.iterrows():
            if cell_is_invalid(row, col):
                nulls += 1

        # Percents are initialized as 0, so we only need to update
        # here if we found invalid rows.
        if nulls > 0:
            percent_missing[col] = 100 * nulls / total
            
    sorted_percents = {k: v for k, v in sorted(percent_missing.items(), key=lambda item: item[1], reverse=True)}

    for k, v in sorted_percents.items():
        print('{:.2f}% of rows missing {} data'.format(v, k))

In [ ]:
get_missing_columns_report(small_plants)

The script below continues the exploration of missing data in the table. However, instead of looking through the lens of column values, we take a look at how many rows are missing a valid value for at least one column. This may prove to be less useful than the exploration above, however it's an easy and quick question to answer, so why not.

Out of the 19559 total rows in the table, only 381 rows (1.95%) contain a valid value for every column in the data.

In [ ]:
def get_rows_with_invalid_data(df):
    # Get the number of rows in table so we have
    # the denominator when we calculate percentage below.
    total = df.shape[0]
    
    # Tally the number of rows with invalid data
    # for at least one column.
    missing_row_count = 0

    for index, row in df.iterrows():
        # Flag that will flip to true as soon as we identify
        # any column with invalid data.
        data_missing = False
        for col in df.columns:
            # If this column is invalid, we can break the loop
            # here for the purposes of this exploration, and
            # include this row in the tally.
            if cell_is_invalid(row, col):
                data_missing = True
                break
        if data_missing:
            missing_row_count += 1

            
    print('{} rows missing at least one valid value.'.format(missing_row_count))
    print('{:.2f}% of rows missing at least one valid value.'.format(100 * missing_row_count / total))

In [ ]:
get_rows_with_invalid_data(small_plants)

### Inconsistent/Non-standard Data
It's not surprising that the table contains a lot of inconsistent or non-standard data, considering the FERC Form 1 allows groups to provide free text information. For fields like plant names, it's expected for values to be different and even unique in most cases. However, for a field like _kind_of_fuel_ in the table, I believe there's a finite and relatively small set of values that are suitable values to provide here. However, because the form allows for the plants to fill in the free text field, in this table we end up with 109 unique values in this column.

Honestly, this is a small enough set that I would create a standardized format for fuel type and manually comb through them to update the table accordingly. For example, something like a pipe-delimited or comma-seperated value that would satisfy describing plants that use any single source of fuel as well as those with combination or hybrid approaches.

So if a plant reports using "Coal/Gas", I might change that value to "COAL|GAS". Similarly any plant that reports "#2 Oil" or "#2" would end up having identical values in the new created. I am not sure if the numbers are significant here (since there are rows containing "#6" or "No. 6 Oil") or if it'd only be necessary to note that those plants use oil.

As I mentioned, given this table and some more time, I think it'd be simple enough to manually create the mapping of old values to new standardized values in transforming this table. However, if we anticipate needing to run the table transformation frequently and on larger sets of data with minimal manual adjustments, it might be worth it to write a module that can attempt automating the transformations. I've begun to do so in the section below. This single function is far from comprehensive but lays out the vision for the approach.

In [ ]:
# List out the unique values in the kind_of_fuel column
unique_fuel_values = small_plants['kind_of_fuel'].unique()
print('Out of 19559 rows of data, we found {} unique values for the kind_of_fuel column:'.format(len(unique_fuel_values)))
for value in unique_fuel_values:
    if value.strip():
        print(value)

In [ ]:
def check_fuel_type(original_value, new_value, fuel_type, standard):
    if fuel_type in original_value:
        return new_value + standard
    return new_value

def standardize_fuel(value):
    # Normalize the original value first
    value = value.strip().lower()
    
    # Any of these values signal that we don't
    # need to inspect the data any more.
    na_list = ['n/a', 'na', 'n.a.']
    if value == 'none' or value in na_list or not value:
        return 'NONE'

    # The chunk below looks for keywords in the original value
    # supplied on the form, and creates a new standardized value
    # that captures any and all fuel types referenced.
    
    # Making an assumption that the Oil/Gas # does not matter for now,
    # but could easily update the pattern matching if that information
    # needs to be captured/preserved. Honestly, I'd default to preserving
    # that information but in the interest of time/simplicity, for now I'm
    # omitting it.
    standardized_fuel = check_fuel_type(value, '', 'oil', 'OIL|')
    standardized_fuel = check_fuel_type(value, standardized_fuel, 'gas', 'GAS|')
    standardized_fuel = check_fuel_type(value, standardized_fuel, 'diesel', 'DIESEL|')
    standardized_fuel = check_fuel_type(value, standardized_fuel, 'hydro', 'HYDRO|')
    standardized_fuel = check_fuel_type(value, standardized_fuel, 'coal', 'COAL|')
    standardized_fuel = check_fuel_type(value, standardized_fuel, 'water', 'WATER|')
    standardized_fuel = check_fuel_type(value, standardized_fuel, 'wind', 'WIND|')
    standardized_fuel = check_fuel_type(value, standardized_fuel, 'solar', 'SOLAR|')
    standardized_fuel = check_fuel_type(value, standardized_fuel, 'waste heat', 'WASTE HEAT|')
    standardized_fuel = check_fuel_type(value, standardized_fuel, 'propane', 'PROPANE|')
    standardized_fuel = check_fuel_type(value, standardized_fuel, 'methane', 'METHANE|')
    standardized_fuel = check_fuel_type(value, standardized_fuel, 'fossil', 'FOSSIL|')
    standardized_fuel = check_fuel_type(value, standardized_fuel, 'river', 'RIVER|')
    standardized_fuel = check_fuel_type(value, standardized_fuel, 'rv', 'RV|')
    standardized_fuel = check_fuel_type(value, standardized_fuel, 'steam', 'STEAM|')
    return standardized_fuel

def standardize_values(df):
    fuel_values = df['kind_of_fuel'].unique()
    print('Original\tStandardized')
    for value in fuel_values:
        standardized_value = standardize_fuel(value)
        print('{}\t{}'.format(value, standardized_value))
        
# Below you can see a side by side comparison of original form values
# and what the updated standardized value would be ()
standardize_values(small_plants)

## Cleaning the data
### Standardize Values
The first effort in the cleaning process is to standardize values in the columns where it would be useful. As mentioned above, I outlined _kind_of_fuel_ as a column which could likely be standardized. So in the code snippet below, we use that standardization process to update those values in our data frame.

### Remove Extraneous Rows
Secondly, I want to remove rows that aren't useful for the analysis which I'm assuming consumers of this table would want to conduct. In the case of this table, I noted earlier that the rows of most significance appear to be those with a valid _yr_constructed_ value. Those seem to be the rows which actually correspond to a small plant, and the rows which our consumers are likely most interested in. However, I also noted that some of the extraneous rows do contain important information that would be useful to apply to the small plant rows which might be missing some values. If this were a real exercise, before removing extraneous rows, I would salvage the useful data from them, supplement the real rows that we're keeping with the salvaged data (e.g. removing a row that introduces a set of "Hydro" plants, but making sure that the corresponding plant rows contain "hydro" in their _kind_of_fuel_ column). Given the short time window, I'm simply removing these rows for not as a prototype. Inline comments describe the same decision/thought process as well.

In [ ]:
# Consts PATTERN and YEAR_CONSTRUCTED_COL come from the
# missing data/invalid data section above. Copied here for convenience:
# YEAR_CONSTRUCTED_COL = 'yr_constructed'
# PATTERN = re.compile("^[0-9]{4}$")

def row_represents_plant(row):
    year_constructed = row[YEAR_CONSTRUCTED_COL].strip()
    return PATTERN.match(year_constructed)
    
def clean_data_frame(df):
    indexes_for_deletion = []
    for index, row in df.iterrows():
        # Mark rows with invalid years for deletion
        # NOTE: In the interest of staying within the 2-4 hour window for the assignment,
        # I did not have time to complete the proposal I noted above in the "Extraneous Data"
        # section. The proposal was to look for rows that act as a heading for rows that
        # follow it, where those heading rows often contain information about the fuel type
        # for rows that follow it. This script simply removes all invalid rows to clean up
        # the table for usability, however, given more time and in the real world, I'd salvage
        # information from those rows before discarding.
        if not row_represents_plant(row):
            indexes_for_deletion.append(index)
        
        # Standardadize fuel values
        df.at[index,'kind_of_fuel'] = standardize_fuel(df.at[index,'kind_of_fuel'])
  
    df = df.drop(df.index[indexes_for_deletion])
    return df

small_plants_copy = small_plants.copy()
small_plants_copy = clean_data_frame(small_plants_copy)

In [ ]:
small_plants_copy.head(60)